# Kaleidocycle Analysis and Animation Demo


In [ ]:

%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os,sys
# Add parent src directory to path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../src')))

# Import kaleidocycle functions
from kaleidocycle import (
    # Geometry and topology
    Kaleidocycle,
    KaleidocycleAnimation,
    optimize_cycle,
    random_hinges,
    normalize_hinges,
    binormals_to_tangents,
    tangents_to_curve,
    pairwise_cosines,
    mean_cosine,
    writhe,
    is_oriented,
    total_twist,
    total_twist_from_curve,
    pairwise_curvature,
    compute_axis,
    compute_torsion,
    curve_to_binormals,
    curve_to_tangents,
    # Animation
    generate_animation,
    clean_animation_frames,
    align_animation_frames,
    sine_gordon_step,
    plot_vertex_values,
    # Constraints
    ConstraintConfig,
    import_json,
    import_csv,
    format_report,
    constraint_residuals,
    plot_hinges,
    plot_band,
)

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Load kaleidocycle data from JSON
filename = 'kaleidocycle_k8_nonoriented_neg_mean_cos.json'
#filename = 'kaleidocycle_k9_oriented_lk4_bending.json'
try:
    if filename.lower().endswith('.json'):
        kc = import_json(filename)
        config = ConstraintConfig(oriented=kc.oriented, enforce_anchors=False, constant_torsion=True, alignment=True, reference_torsion=kc.mean_cosine)
        binormals = kc.hinges
    else:
        binormals = import_csv(filename)
        config = ConstraintConfig(oriented=is_oriented(binormals), enforce_anchors=False, constant_torsion=True, alignment=True)
except Exception as e:
    print(f"Error loading file {filename}: {e}")
    kc = Kaleidocycle(8, oriented=False)
    binormals = kc.hinges
    config = kc.config
print(format_report(binormals, config))

In [ ]:
num_frames = 100
step_size = 0.05
rule = "mKdV2" # "sine_gordon" # "step" #

config = kc.config
frames = generate_animation(kc.hinges, num_frames=num_frames, step_size=step_size, rule=rule, oriented=kc.oriented, config=config)

# Create animation
anim = KaleidocycleAnimation(
    frames=frames,
    evolution_rule=rule,
    metadata={"step_size": step_size}
)

# align frames
anim.align()

# Compute properties
anim.compute_vertex_property("curvature")
anim.compute_scalar_property("penalty")

plt.close('all')
plt.plot(anim.scalar_properties["penalty"], marker='o')
plt.xlabel("Frame")
plt.ylabel("Penalty")
plt.title("Penalty Evolution")
plt.grid()
plt.show()


In [ ]:
fig, ani = plot_band(
      animation=anim,
      scalar_properties=["penalty", "linking_number"],
  )

## Cleaning and Aligning Animation Frames

We can filter out infeasible frames and remove spurious global rotations.

In [ ]:
# Clean frames
cleaned, kept_indices = clean_animation_frames(
    frames,
    config=config,
    tolerance=10.0,  # Permissive tolerance
    fix_twist=True,
    check_torsion=False,
)

print(f"Frames after cleaning: {len(cleaned)}/{len(frames)}")
print(f"Kept indices: {kept_indices[:10]}...")

# Align frames to remove global rotation
aligned = align_animation_frames(cleaned, use_barycentre=True)
print(f"Frames aligned: {len(aligned)}")

# Measure alignment quality
var_before = np.std([np.linalg.norm(f - cleaned[0]) for f in cleaned])
var_after = np.std([np.linalg.norm(f - aligned[0]) for f in aligned])

print(f"\nVariation before alignment: {var_before:.4f}")
print(f"Variation after alignment:  {var_after:.4f}")
print(f"Improvement: {(1 - var_after/var_before)*100:.1f}%")

In [ ]:
anim_aligned = KaleidocycleAnimation(frames=aligned)
fig, ani = plot_band(
      animation=anim_aligned,
      scalar_properties=["penalty", "linking_number"],
  )